In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as Data
import math
torch.manual_seed(8)
import time
import numpy as np
import gc
import sys
sys.setrecursionlimit(50000)
import pickle
torch.backends.cudnn.benchmark = True
torch.set_default_tensor_type('torch.cuda.FloatTensor')
# from tensorboardX import SummaryWriter
torch.nn.Module.dump_patches = True
import copy
import pandas as pd
#then import my own modules
from AttentiveFP.AttentiveLayers_Sim_copy import Fingerprint, GRN, AFSE
from AttentiveFP import Fingerprint_viz, save_smiles_dicts, get_smiles_dicts, get_smiles_array, moltosvg_highlight

In [2]:
from rdkit import Chem
# from rdkit.Chem import AllChem
from rdkit.Chem import QED
from rdkit.Chem import rdMolDescriptors, MolSurf
from rdkit.Chem.Draw import SimilarityMaps
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdDepictor
from rdkit.Chem.Draw import rdMolDraw2D
%matplotlib inline
from numpy.polynomial.polynomial import polyfit
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib
import seaborn as sns; sns.set()
from IPython.display import SVG, display
import sascorer
from AttentiveFP.utils import EarlyStopping
from AttentiveFP.utils import Meter
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
import AttentiveFP.Featurizer
import scipy

In [3]:
train_filename = "./data/benchmark/EC50_Q99500_1_100_train.csv"
test_filename = "./data/benchmark/EC50_Q99500_1_100_test.csv"
test_active = 100
val_rate = 0.2
random_seed = 68
file_list1 = train_filename.split('/')
file1 = file_list1[-1]
file1 = file1[:-10]
number = '_run_0'
model_file = "model_file/3_GAFSE_"+file1+number
log_dir = f'log/{"3_GAFSE_"+file1}'+number
result_dir = './result/3_GAFSE_'+file1+number
print(file1)
print(model_file)

EC50_Q99500_1_100
model_file/3_GAFSE_EC50_Q99500_1_100_run_0


In [4]:
# task_name = 'Malaria Bioactivity'
tasks = ['value']

# train_filename = "../data/active_inactive/median_active/EC50/Q99500.csv"
feature_filename = train_filename.replace('.csv','.pickle')
filename = train_filename.replace('.csv','')
prefix_filename = train_filename.split('/')[-1].replace('.csv','')
train_df = pd.read_csv(train_filename, header=0, names = ["smiles","value"],usecols=[0,1])
# train_df = train_df[1:]
# train_df = train_df.drop(0,axis=1,inplace=False) 
print(train_df[:5])
# print(train_df.iloc(1))
def add_canonical_smiles(train_df):
    smilesList = train_df.smiles.values
    print("number of all smiles: ",len(smilesList))
    atom_num_dist = []
    remained_smiles = []
    canonical_smiles_list = []
    for smiles in smilesList:
        try:        
            mol = Chem.MolFromSmiles(smiles)
            atom_num_dist.append(len(mol.GetAtoms()))
            remained_smiles.append(smiles)
            canonical_smiles_list.append(Chem.MolToSmiles(Chem.MolFromSmiles(smiles), isomericSmiles=True))
        except:
            print(smiles)
            pass
    print("number of successfully processed smiles: ", len(remained_smiles))
    train_df = train_df[train_df["smiles"].isin(remained_smiles)]
    train_df['cano_smiles'] =canonical_smiles_list
    return train_df
# print(train_df)
train_df = add_canonical_smiles(train_df)

print(train_df.head())
# plt.figure(figsize=(5, 3))
# sns.set(font_scale=1.5)
# ax = sns.distplot(atom_num_dist, bins=28, kde=False)
# plt.tight_layout()
# # plt.savefig("atom_num_dist_"+prefix_filename+".png",dpi=200)
# plt.show()
# plt.close()


                                              smiles     value
0  CCCC1=NC=C(C=C1)C2=NC(=NO2)C3=CC(=C(C(=C3)CC)O... -3.247973
1  C1=CC=C(C=C1)C2=C(SC(=C2)C3=NC(=NO3)C4=CC=C(C=... -3.200014
2  CCC1=CC(=CC(=C1OCC(CNC(=O)CO)O)C)C2=NOC(=N2)C3... -1.653213
3  CCC1=CC(=CC(=C1OCC(CNC(=O)CO)O)C)C2=NOC(=N2)C3... -3.643453
4    CC1=CC(=NN1C2=NC(=C3C4=C(CCCC4)SC3=N2)NCC(C)O)C -3.891537
number of all smiles:  545
number of successfully processed smiles:  545
                                              smiles     value  \
0  CCCC1=NC=C(C=C1)C2=NC(=NO2)C3=CC(=C(C(=C3)CC)O... -3.247973   
1  C1=CC=C(C=C1)C2=C(SC(=C2)C3=NC(=NO3)C4=CC=C(C=... -3.200014   
2  CCC1=CC(=CC(=C1OCC(CNC(=O)CO)O)C)C2=NOC(=N2)C3... -1.653213   
3  CCC1=CC(=CC(=C1OCC(CNC(=O)CO)O)C)C2=NOC(=N2)C3... -3.643453   
4    CC1=CC(=NN1C2=NC(=C3C4=C(CCCC4)SC3=N2)NCC(C)O)C -3.891537   

                                         cano_smiles  
0  CCCc1ccc(-c2nc(-c3cc(C)c(OCC(O)CNC(=O)CO)c(CC)...  
1  O=C([O-])c1cnn(Cc2ccc(-c3noc(-c4cc(-c5c

In [5]:
start_time = str(time.ctime()).replace(':','-').replace(' ','_')

p_dropout= 0.03
fingerprint_dim = 100

weight_decay = 4.3 # also known as l2_regularization_lambda
learning_rate = 4
radius = 2 # default: 2
T = 1
per_task_output_units_num = 1 # for regression model
output_units_num = len(tasks) * per_task_output_units_num

In [6]:
test_df = pd.read_csv(test_filename,header=0,names=["smiles","value"],usecols=[0,1])
test_df = add_canonical_smiles(test_df)
for l in test_df["cano_smiles"]:
    if l in train_df["cano_smiles"]:
        print("same smiles:",l)
        
print(test_df.shape)
print(test_df.head())

number of all smiles:  1053
number of successfully processed smiles:  1053
(1053, 3)
                                              smiles     value  \
0  CC(C)C1=CC=C(C=C1)CC2=NC(=CO2)C3=CC=C(C=C3)CCC... -2.966142   
1  C1CC1C2=C(ON=C2C3=NC(=NO3)C4=CC=C(C=C4)CN5CC(C... -3.255273   
2  CC1=CC(=CC(=C1OCCCNCC(=O)O)C)CCC(=O)C2=C3CC4C(... -2.303196   
3  C1=CC(=CC=C1CCC(CO)(COP(=O)(O)O)N)C2=CN(N=N2)C... -3.086004   
4  CCCC1=CC=C(C=C1)C2=C3COC4=C(C3=NO2)C=CC(=C4)CN... -3.913814   

                                         cano_smiles  
0  CC(C)c1ccc(Cc2nc(-c3ccc(CCC(N)(CO)COP(=O)(O)O)...  
1  O=C(O)C1CN(Cc2ccc(-c3noc(-c4noc(-c5ccccc5)c4C4...  
2  Cc1cc(CCC(=O)c2sc(C)c3c2CC2C3C2(C)C)cc(C)c1OCC...  
3  NC(CO)(CCc1ccc(-c2cn(-c3ccc(OC(F)(F)F)cc3)nn2)...  
4  CCCc1ccc(-c2onc3c2COc2cc(CN4CC(C(=O)O)C4)ccc2-...  


In [7]:
print(feature_filename)
print(filename)
total_df = pd.concat([train_df,test_df],axis=0)
total_smilesList = total_df['smiles'].values
print(len(total_smilesList))
# if os.path.isfile(feature_filename):
#     feature_dicts = pickle.load(open(feature_filename, "rb" ))
# else:
#     feature_dicts = save_smiles_dicts(smilesList,filename)
feature_dicts = save_smiles_dicts(total_smilesList,filename)
remained_df = total_df[total_df["cano_smiles"].isin(feature_dicts['smiles_to_atom_mask'].keys())]
uncovered_df = total_df.drop(remained_df.index)

./data/benchmark/EC50_Q99500_1_100_train.pickle
./data/benchmark/EC50_Q99500_1_100_train
1598
feature dicts file saved as ./data/benchmark/EC50_Q99500_1_100_train.pickle


In [8]:
val_df = train_df.sample(frac=val_rate,random_state=random_seed)
train_df = train_df.drop(val_df.index)
train_df = train_df.reset_index(drop=True)
train_df = train_df[train_df["cano_smiles"].isin(feature_dicts['smiles_to_atom_mask'].keys())]
train_df = train_df.reset_index(drop=True)
val_df = val_df[val_df["cano_smiles"].isin(feature_dicts['smiles_to_atom_mask'].keys())]
val_df = val_df.reset_index(drop=True)
test_df = test_df[test_df["cano_smiles"].isin(feature_dicts['smiles_to_atom_mask'].keys())]
test_df = test_df.reset_index(drop=True)
print(train_df.shape,val_df.shape,test_df.shape)

(436, 3) (109, 3) (1053, 3)


In [9]:
x_atom, x_bonds, x_atom_index, x_bond_index, x_mask, smiles_to_rdkit_list = get_smiles_array([total_df["cano_smiles"].values[0]],feature_dicts)
num_atom_features = x_atom.shape[-1]
num_bond_features = x_bonds.shape[-1]
loss_function = nn.MSELoss()
model = Fingerprint(radius, T, num_atom_features, num_bond_features,
            fingerprint_dim, output_units_num, p_dropout)
amodel = AFSE(fingerprint_dim, output_units_num, p_dropout)
gmodel = GRN(radius, T, num_atom_features, num_bond_features,
            fingerprint_dim, p_dropout)
model.cuda()
amodel.cuda()
gmodel.cuda()

# optimizer = optim.Adam([
# {'params': model.parameters(), 'lr': 10**(-learning_rate), 'weight_decay ': 10**-weight_decay}, 
# {'params': gmodel.parameters(), 'lr': 10**(-learning_rate), 'weight_decay ': 10**-weight_decay}, 
# ])

optimizer = optim.Adam(params=model.parameters(), lr=10**(-learning_rate), weight_decay=10**-weight_decay)

optimizer_AFSE = optim.Adam(params=amodel.parameters(), lr=10**(-learning_rate), weight_decay=10**-weight_decay)

# optimizer_AFSE = optim.SGD(params=amodel.parameters(), lr = 0.01, momentum=0.9)

optimizer_GRN = optim.Adam(params=gmodel.parameters(), lr=10**(-learning_rate), weight_decay=10**-weight_decay)

# tensorboard = SummaryWriter(log_dir="runs/"+start_time+"_"+prefix_filename+"_"+str(fingerprint_dim)+"_"+str(p_dropout))

model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
# print(params)
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print(name, param.data.shape)
        

In [10]:
import numpy as np
from matplotlib import pyplot as plt

def sorted_show_pik(dataset, p, k, k_predict, i, acc):
    p_value = dataset[tasks[0]].astype(float).tolist()
    x = np.arange(0,len(dataset),1)
#     print('plt',dataset.head(),p[:10],k_predict,k)
#     plt.figure()
#     fig, ax1 = plt.subplots()
#     ax1.grid(False)
#     ax2 = ax1.twinx()
#     plt.grid(False)
    plt.scatter(x,p,marker='.',s=6,color='r',label='predict')
#     plt.ylabel('predict')
    plt.scatter(x,p_value,s=6,marker=',',color='blue',label='p_value')
    plt.axvline(x=k-1,ls="-",c="black")#添加垂直直线
    k_value = np.ones(len(dataset))
# #     print(EC50[k-1])
    k_value = k_value*k_predict
    plt.plot(x,k_value,'-',color='black')
    plt.ylabel('p_value')
    plt.title("epoch: {},  top-k recall: {}".format(i,acc))
    plt.legend(loc=3,fontsize=5)
    plt.show()
    

def topk_acc2(df, predict, k, active_num, show_flag=False, i=0):
    df['predict'] = predict
    df2 = df.sort_values(by='predict',ascending=False) # 拼接预测值后对预测值进行排序
#     print('df2:\n',df2)
    
    df3 = df2[:k]  #取按预测值排完序后的前k个
    
    true_sort = df.sort_values(by=tasks[0],ascending=False) #返回一个新的按真实值排序列表
    k_true = true_sort[tasks[0]].values[k-1]  # 真实排第k个的活性值
#     print('df3:\n',df3['predict'])
#     print('k_true: ',type(k_true),k_true)
#     print('k_true: ',k_true,'min_predict: ',df3['predict'].values[-1],'index: ',df3['predict'].values>=k_true,'acc_num: ',len(df3[df3['predict'].values>=k_true]),
#           'fp_num: ',len(df3[df3['predict'].values>=-4.1]),'k: ',k)
    acc = len(df3[df3[tasks[0]].values>=k_true])/k #预测值前k个中真实排在前k个的个数/k
    fp = len(df3[df3[tasks[0]].values==-4.1])/k  #预测值前k个中为-4.1的个数/k
    if k>active_num:
        min_active = true_sort[tasks[0]].values[active_num-1]
        acc = len(df3[df3[tasks[0]].values>=min_active])/k
    
    if(show_flag):
        #进来的是按实际活性值排好序的
        sorted_show_pik(true_sort,true_sort['predict'],k,k_predict,i,acc)
    return acc,fp

def topk_recall(df, predict, k, active_num, show_flag=False, i=0):
    df['predict'] = predict
    df2 = df.sort_values(by='predict',ascending=False) # 拼接预测值后对预测值进行排序
#     print('df2:\n',df2)
        
    df3 = df2[:k]  #取按预测值排完序后的前k个，因为后面的全是-4.1
    
    true_sort = df.sort_values(by=tasks[0],ascending=False) #返回一个新的按真实值排序列表
    min_active = true_sort[tasks[0]].values[active_num-1]  # 真实排第k个的活性值
#     print('df3:\n',df3['predict'])
#     print('min_active: ',type(min_active),min_active)
#     print('min_active: ',min_active,'min_predict: ',df3['predict'].values[-1],'index: ',df3['predict'].values>=min_active,'acc_num: ',len(df3[df3['predict'].values>=min_active]),
#           'fp_num: ',len(df3[df3['predict'].values>=-4.1]),'k: ',k,'active_num: ',active_num)
    acc = len(df3[df3[tasks[0]].values>-4.1])/active_num #预测值前k个中真实排在前active_num个的个数/active_num
    fp = len(df3[df3[tasks[0]].values==-4.1])/k  #预测值前k个中为-4.1的个数/active_num
    
    if(show_flag):
        #进来的是按实际活性值排好序的
        sorted_show_pik(true_sort,true_sort['predict'],k,k_predict,i,acc)
    return acc,fp

    
def topk_acc_recall(df, predict, k, active_num, show_flag=False, i=0):
    if k>active_num:
        return topk_recall(df, predict, k, active_num, show_flag, i)
    return topk_acc2(df,predict,k, active_num,show_flag,i)

def weighted_top_index(df, predict, active_num):
    weighted_acc_list=[]
    for k in np.arange(1,len(df)+1,1):
        acc, fp = topk_acc_recall(df, predict, k, active_num)
        weight = (len(df)-k)/len(df)
#         print('weight=',weight,'acc=',acc)
        weighted_acc_list.append(acc*weight)#
    weighted_acc_list = np.array(weighted_acc_list)
#     print('weighted_acc_list=',weighted_acc_list)
    return np.sum(weighted_acc_list)/weighted_acc_list.shape[0]

def AP(df, predict, active_num):
    prec = []
    rec = []
    for k in np.arange(1,len(df)+1,1):
        prec_k, fp1 = topk_acc2(df,predict,k, active_num)
        rec_k, fp2 = topk_recall(df, predict, k, active_num)
        prec.append(prec_k)
        rec.append(rec_k)
    # 取所有不同的recall对应的点处的精度值做平均
    # first append sentinel values at the end
    mrec = np.concatenate(([0.], rec, [1.]))
    mpre = np.concatenate(([0.], prec, [0.]))

    # 计算包络线，从后往前取最大保证precise非减
    for i in range(mpre.size - 1, 0, -1):
        mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

    # 找出所有检测结果中recall不同的点
    i = np.where(mrec[1:] != mrec[:-1])[0]
#     print(prec)
#     print('prec='+str(prec)+'\n\n'+'rec='+str(rec))

    # and sum (\Delta recall) * prec
    # 用recall的间隔对精度作加权平均
    ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap

In [11]:
def caculate_r2(y,predict):
#     print(y)
#     print(predict)
    y = torch.FloatTensor(y).reshape(-1,1)
    predict = torch.FloatTensor(predict).reshape(-1,1)
    y_mean = torch.mean(y)
    predict_mean = torch.mean(predict)
    
    y1 = torch.pow(torch.mm((y-y_mean).t(),(predict-predict_mean)),2)
    y2 = torch.mm((y-y_mean).t(),(y-y_mean))*torch.mm((predict-predict_mean).t(),(predict-predict_mean))
#     print(y1,y2)
    return y1/y2

In [12]:
from torch.autograd import Variable
def l2_norm(input, dim):
    norm = torch.norm(input, dim=dim, keepdim=True)
    output = torch.div(input, norm+1e-6)
    return output

def normalize_perturbation(d,dim=-1):
    output = l2_norm(d, dim)
    return output

def tanh(x):
    return (torch.exp(x)-torch.exp(-x))/(torch.exp(x)+torch.exp(-x))

def sigmoid(x):
    return 1/(1+torch.exp(-x))

def perturb_feature(f, model, alpha=1, lamda=10**-learning_rate, output_lr=False, output_plr=False, y=None):
    mol_prediction = model(feature=f, d=0)
    pred = mol_prediction.detach()
#     f = torch.div(f, torch.norm(f, dim=-1, keepdim=True)+1e-9)
    eps = 1e-6 * normalize_perturbation(torch.randn(f.shape))
    eps = Variable(eps, requires_grad=True)
    # Predict on randomly perturbed image
    eps_p = model(feature=f, d=eps.cuda())
    eps_p_ = model(feature=f, d=-eps.cuda())
    p_aux = nn.Sigmoid()(eps_p/(pred+1e-6))
    p_aux_ = nn.Sigmoid()(eps_p_/(pred+1e-6))
#     loss = nn.BCELoss()(abs(p_aux),torch.ones_like(p_aux))+nn.BCELoss()(abs(p_aux_),torch.ones_like(p_aux_))
    loss = loss_function(p_aux,torch.ones_like(p_aux))+loss_function(p_aux_,torch.ones_like(p_aux_))
    loss.backward(retain_graph=True)

    # Based on perturbed image, get direction of greatest error
    eps_adv = eps.grad#/10**-learning_rate
    optimizer_AFSE.zero_grad()
    # Use that direction as adversarial perturbation
    eps_adv_normed = normalize_perturbation(eps_adv)
    d_adv = lamda * eps_adv_normed.cuda()
    if output_lr:
        f_p, max_lr = model(feature=f, d=d_adv, output_lr=output_lr)
    f_p = model(feature=f, d=d_adv)
    f_p_ = model(feature=f, d=-d_adv)
    p = nn.Sigmoid()(f_p/(pred+1e-6))
    p_ = nn.Sigmoid()(f_p_/(pred+1e-6))
    vat_loss = loss_function(p,torch.ones_like(p))+loss_function(p_,torch.ones_like(p_))
    if output_lr:
        if output_plr:
            loss = loss_function(mol_prediction,y)
            loss.backward(retain_graph=True)
            optimizer_AFSE.zero_grad()
            punish_lr = torch.norm(torch.mean(eps.grad,0))
            return eps_adv, d_adv, vat_loss, mol_prediction, max_lr, punish_lr
        return eps_adv, d_adv, vat_loss, mol_prediction, max_lr
    return eps_adv, d_adv, vat_loss, mol_prediction

def mol_with_atom_index( mol ):
    atoms = mol.GetNumAtoms()
    for idx in range( atoms ):
        mol.GetAtomWithIdx( idx ).SetProp( 'molAtomMapNumber', str( mol.GetAtomWithIdx( idx ).GetIdx() ) )
    return mol

def d_loss(f, pred, model, y_val):
    diff_loss = 0
    length = len(pred)
    for i in range(length):
        for j in range(length):
            if j == i:
                continue
            pred_diff = model(feature_only=True, feature1=f[i], feature2=f[j])
            true_diff = y_val[i] - y_val[j]
            diff_loss += loss_function(pred_diff, torch.Tensor([true_diff]).view(-1,1))
    diff_loss = diff_loss/(length*(length-1))
    return diff_loss

In [13]:
def CE(x,y):
    c = 0
    l = len(y)
    for i in range(l):
        if y[i]==1:
            c += 1
    w1 = (l-c)/l
    w0 = c/l
    loss = -w1*y*torch.log(x+1e-6)-w0*(1-y)*torch.log(1-x+1e-6)
    loss = loss.mean(-1)
    return loss

def weighted_CE_loss(x,y):
    weight = 1/(y.detach().float().mean(0)+1e-9)
    weighted_CE = nn.CrossEntropyLoss(weight=weight)
#     atom_weights = (atom_weights-min(atom_weights))/(max(atom_weights)-min(atom_weights))
    return weighted_CE(x, torch.argmax(y,-1))

def py_sigmoid_focal_loss(pred,
                          target,
                          weight=None,
                          gamma=2.0,
                          alpha=0.25):
    weighted_CE = nn.CrossEntropyLoss(weight=alpha, reduce=False)
    focal_weight = (1-torch.max(pred * target, -1)[0])**gamma
    loss = focal_weight * weighted_CE(pred, torch.argmax(target,-1))
    loss = torch.mean(loss)
    return loss

def generate_loss_function(refer_atom_list, x_atom, refer_bond_list, bond_neighbor, validity_mask, atom_list, bond_list):
    [a,b,c] = x_atom.shape
    [d,e,f,g] = bond_neighbor.shape
    ce_loss = nn.CrossEntropyLoss()
    one_hot_loss = 0
    run_times = 0
    validity_mask = torch.from_numpy(validity_mask).cuda()
    for i in range(a):
        l = (x_atom[i].sum(-1)!=0).sum(-1)
        atom_weights = 1-x_atom[i,:l,:16].sum(-2)/l
        ce_atom_loss = nn.CrossEntropyLoss(weight=atom_weights)
#         print(atom_weights[1], refer_atom_list[i,0,torch.argmax(x_atom[i,0,:16],-1)], torch.argmax(x_atom[i,0,:16],-1))
#         one_hot_loss += ce_atom_loss(refer_atom_list[i,:l,:16], torch.argmax(x_atom[i,:l,:16],-1))- \
#                          (((validity_mask[i,:l]*torch.log(1-atom_list[i,:l,:16]+1e-9)).sum(-1)/(validity_mask[i,:l].sum(-1)+1e-9))).mean(-1)
        one_hot_loss += py_sigmoid_focal_loss(refer_atom_list[i,:l,:16], x_atom[i,:l,:16], alpha=atom_weights)- \
                          (((validity_mask[i,:l]*torch.log(1-atom_list[i,:l,:16]+1e-9)).sum(-1)/(validity_mask[i,:l].sum(-1)+1e-9))).mean(-1)
        run_times += 2
    total_loss = one_hot_loss/run_times
    return total_loss, 0, 0, 0


def train(model, amodel, gmodel, dataset, test_df, optimizer_list, loss_function, epoch):
    model.train()
    amodel.train()
    gmodel.train()
    optimizer, optimizer_AFSE, optimizer_GRN = optimizer_list
    np.random.seed(epoch)
    max_len = np.max([len(dataset),len(test_df)])
    valList = np.arange(0,max_len)
    #shuffle them
    np.random.shuffle(valList)
    batch_list = []
    for i in range(0, max_len, batch_size):
        batch = valList[i:i+batch_size]
        batch_list.append(batch)
    for counter, batch in enumerate(batch_list):
        batch_df = dataset.loc[batch%len(dataset),:]
        batch_test = test_df.loc[batch%len(test_df),:]
        global_step = epoch * len(batch_list) + counter
        smiles_list = batch_df.cano_smiles.values
        smiles_list_test = batch_test.cano_smiles.values
        y_val = batch_df[tasks[0]].values.astype(float)
        
        x_atom, x_bonds, x_atom_index, x_bond_index, x_mask, smiles_to_rdkit_list = get_smiles_array(smiles_list,feature_dicts)
        x_atom_test, x_bonds_test, x_atom_index_test, x_bond_index_test, x_mask_test, smiles_to_rdkit_list_test = get_smiles_array(smiles_list_test,feature_dicts)
        activated_features, mol_feature = model(torch.Tensor(x_atom),torch.Tensor(x_bonds),torch.cuda.LongTensor(x_atom_index),
                                                torch.cuda.LongTensor(x_bond_index),torch.Tensor(x_mask),output_activated_features=True)
#         mol_feature = torch.div(mol_feature, torch.norm(mol_feature, dim=-1, keepdim=True)+1e-9)
#         activated_features = torch.div(activated_features, torch.norm(activated_features, dim=-1, keepdim=True)+1e-9)
        refer_atom_list, refer_bond_list = gmodel(torch.Tensor(x_atom),torch.Tensor(x_bonds),torch.cuda.LongTensor(x_atom_index),
                                                  torch.cuda.LongTensor(x_bond_index),torch.Tensor(x_mask),
                                                  mol_feature=mol_feature,activated_features=activated_features.detach())
        
        x_atom = torch.Tensor(x_atom)
        x_bonds = torch.Tensor(x_bonds)
        x_bond_index = torch.cuda.LongTensor(x_bond_index)
        
        bond_neighbor = [x_bonds[i][x_bond_index[i]] for i in range(len(batch_df))]
        bond_neighbor = torch.stack(bond_neighbor, dim=0)
        
        eps_adv, d_adv, vat_loss, mol_prediction, conv_lr, punish_lr = perturb_feature(mol_feature, amodel, alpha=1, 
                                                                                       lamda=10**-learning_rate, output_lr=True, 
                                                                                       output_plr=True, y=torch.Tensor(y_val).view(-1,1)) # 10**-learning_rate     
        regression_loss = loss_function(mol_prediction, torch.Tensor(y_val).view(-1,1))
        atom_list, bond_list = gmodel(torch.Tensor(x_atom),torch.Tensor(x_bonds),torch.cuda.LongTensor(x_atom_index),torch.cuda.LongTensor(x_bond_index),
                                      torch.Tensor(x_mask),mol_feature=mol_feature+d_adv/1e-6,activated_features=activated_features.detach())
        success_smiles_batch, modified_smiles, success_batch, total_batch, reconstruction, validity, validity_mask = modify_atoms(smiles_list, x_atom, 
                            bond_neighbor, atom_list, bond_list,smiles_list,smiles_to_rdkit_list,
                                                     refer_atom_list, refer_bond_list,topn=1)
        reconstruction_loss, one_hot_loss, interger_loss,binary_loss = generate_loss_function(refer_atom_list, x_atom, refer_bond_list, 
                                                                                              bond_neighbor, validity_mask, atom_list, 
                                                                                              bond_list)
        x_atom_test = torch.Tensor(x_atom_test)
        x_bonds_test = torch.Tensor(x_bonds_test)
        x_bond_index_test = torch.cuda.LongTensor(x_bond_index_test)
        
        bond_neighbor_test = [x_bonds_test[i][x_bond_index_test[i]] for i in range(len(batch_test))]
        bond_neighbor_test = torch.stack(bond_neighbor_test, dim=0)
        activated_features_test, mol_feature_test = model(torch.Tensor(x_atom_test),torch.Tensor(x_bonds_test),
                                                          torch.cuda.LongTensor(x_atom_index_test),torch.cuda.LongTensor(x_bond_index_test),
                                                          torch.Tensor(x_mask_test),output_activated_features=True)
#         mol_feature_test = torch.div(mol_feature_test, torch.norm(mol_feature_test, dim=-1, keepdim=True)+1e-9)
#         activated_features_test = torch.div(activated_features_test, torch.norm(activated_features_test, dim=-1, keepdim=True)+1e-9)
        eps_test, d_test, test_vat_loss, mol_prediction_test = perturb_feature(mol_feature_test, amodel, 
                                                                                    alpha=1, lamda=10**-learning_rate)
        atom_list_test, bond_list_test = gmodel(torch.Tensor(x_atom_test),torch.Tensor(x_bonds_test),torch.cuda.LongTensor(x_atom_index_test),
                                                torch.cuda.LongTensor(x_bond_index_test),torch.Tensor(x_mask_test),
                                                mol_feature=mol_feature_test+d_test/1e-6,activated_features=activated_features_test.detach())
        refer_atom_list_test, refer_bond_list_test = gmodel(torch.Tensor(x_atom_test),torch.Tensor(x_bonds_test),
                                                            torch.cuda.LongTensor(x_atom_index_test),torch.cuda.LongTensor(x_bond_index_test),torch.Tensor(x_mask_test),
                                                            mol_feature=mol_feature_test,activated_features=activated_features_test.detach())
        success_smiles_batch_test, modified_smiles_test, success_batch_test, total_batch_test, reconstruction_test, validity_test, validity_mask_test = modify_atoms(smiles_list_test, x_atom_test, 
                            bond_neighbor_test, atom_list_test, bond_list_test,smiles_list_test,smiles_to_rdkit_list_test,
                                                     refer_atom_list_test, refer_bond_list_test,topn=1)
        test_reconstruction_loss, test_one_hot_loss, test_interger_loss,test_binary_loss = generate_loss_function(atom_list_test, x_atom_test, bond_list_test, bond_neighbor_test, validity_mask_test, atom_list_test, bond_list_test)
        
        if vat_loss>1 or test_vat_loss>1:
            vat_loss = 1*(vat_loss/(vat_loss+1e-6).item())
            test_vat_loss = 1*(test_vat_loss/(test_vat_loss+1e-6).item())
        
        max_lr = 1e-3
        conv_lr = conv_lr - conv_lr**2 + 0.06 * punish_lr
        if conv_lr < max_lr and conv_lr >= 0:
            for param_group in optimizer_AFSE.param_groups:
                param_group["lr"] = conv_lr.detach()
                AFSE_lr = conv_lr    
        elif conv_lr < 0:
            for param_group in optimizer_AFSE.param_groups:
                param_group["lr"] = 0
                AFSE_lr = 0
        elif conv_lr >= max_lr:
            for param_group in optimizer_AFSE.param_groups:
                param_group["lr"] = max_lr
                AFSE_lr = max_lr
        
        logger.add_scalar('loss/regression', regression_loss, global_step)
        logger.add_scalar('loss/AFSE', vat_loss, global_step)
        logger.add_scalar('loss/AFSE_test', test_vat_loss, global_step)
        logger.add_scalar('loss/GRN', reconstruction_loss, global_step)
        logger.add_scalar('loss/GRN_test', test_reconstruction_loss, global_step)
        logger.add_scalar('loss/GRN_one_hot', one_hot_loss, global_step)
        logger.add_scalar('loss/GRN_interger', interger_loss, global_step)
        logger.add_scalar('loss/GRN_binary', binary_loss, global_step)
        logger.add_scalar('lr/max_lr', conv_lr, global_step)
        logger.add_scalar('lr/punish_lr', punish_lr, global_step)
        logger.add_scalar('lr/AFSE_lr', AFSE_lr, global_step)
    
        optimizer.zero_grad()
        optimizer_AFSE.zero_grad()
        optimizer_GRN.zero_grad()
        loss =  regression_loss + 0.6 * (vat_loss + test_vat_loss) + 0.3 * (reconstruction_loss + test_reconstruction_loss)
        loss.backward()
        optimizer.step()
        optimizer_AFSE.step()
        optimizer_GRN.step()

        
def clear_atom_map(mol):
    [a.ClearProp('molAtomMapNumber') for a  in mol.GetAtoms()]
    return mol

def mol_with_atom_index( mol ):
    atoms = mol.GetNumAtoms()
    for idx in range( atoms ):
        mol.GetAtomWithIdx( idx ).SetProp( 'molAtomMapNumber', str( mol.GetAtomWithIdx( idx ).GetIdx() ) )
    return mol
        
def modify_atoms(smiles, x_atom, bond_neighbor, atom_list, bond_list, y_smiles, smiles_to_rdkit_list,refer_atom_list, refer_bond_list,topn=1,viz=False):
    x_atom = x_atom.cpu().detach().numpy()
    bond_neighbor = bond_neighbor.cpu().detach().numpy()
    atom_list = atom_list.cpu().detach().numpy()
    bond_list = bond_list.cpu().detach().numpy()
    refer_atom_list = refer_atom_list.cpu().detach().numpy()
    refer_bond_list = refer_bond_list.cpu().detach().numpy()
    atom_symbol_sorted = np.argsort(x_atom[:,:,:16], axis=-1)
    atom_symbol_generated_sorted = np.argsort(atom_list[:,:,:16], axis=-1)
    generate_confidence_sorted = np.sort(atom_list[:,:,:16], axis=-1)
    modified_smiles = []
    success_smiles = []
    success_reconstruction = 0
    success_validity = 0
    success = [0 for i in range(topn)]
    total = [0 for i in range(topn)]
    confidence_threshold = 0.001
    validity_mask = np.zeros_like(atom_list[:,:,:16])
    symbol_list = ['B','C','N','O','F','Si','P','S','Cl','As','Se','Br','Te','I','At','other']
    symbol_to_rdkit = [4,6,7,8,9,14,15,16,17,33,34,35,52,53,85,0]
    for i in range(len(atom_list)):
        rank = 0
        top_idx = 0
        flag = 0
        first_run_flag = True
        l = (x_atom[i].sum(-1)!=0).sum(-1)
        cano_smiles = Chem.MolToSmiles(Chem.MolFromSmiles(smiles[i]))
        mol = mol_with_atom_index(Chem.MolFromSmiles(smiles[i]))
        counter = 0
        for j in range(l): 
            if mol.GetAtomWithIdx(int(smiles_to_rdkit_list[cano_smiles][j])).GetAtomicNum() == \
                symbol_to_rdkit[refer_atom_list[i,j,:16].argmax(-1)]:
                counter += 1
#             print(f'atom#{smiles_to_rdkit_list[cano_smiles][j]}(f):',{symbol_list[k]: np.around(refer_atom_list[i,j,k],3) for k in range(16)},
#                   f'\natom#{smiles_to_rdkit_list[cano_smiles][j]}(f+d):',{symbol_list[k]: np.around(atom_list[i,j,k],3) for k in range(16)},
#                  '\n------------------------------------------------------------------------------------------------------------')
#         print('预测为每个原子的平均概率：\n',np.around(atom_list[i,:l,:16].mean(1),2))
#         print('预测为每个原子的最大概率：\n',np.around(atom_list[i,:l,:16].max(1),2))
        if counter == l:
            success_reconstruction += 1
        while not flag==topn:
            if rank == 16:
                rank = 0
                top_idx += 1
            if top_idx == l:
#                 print('没有满足条件的分子生成。')
                flag += 1
                continue
#             if np.sum((atom_symbol_sorted[i,:l,-1]!=atom_symbol_generated_sorted[i,:l,-1-rank]).astype(int))==0:
#                 print(f'根据预测的第{rank}大概率的原子构成的分子与原分子一致，原子位重置为0，生成下一个元素……')
#                 rank += 1
#                 top_idx = 0
#                 generate_index = np.argsort((atom_list[i,:l,:16]-refer_atom_list[i,:l,:16] -\
#                                              x_atom[i,:l,:16]).max(-1))[-1-top_idx]
#             print('i:',i,'top_idx:', top_idx, 'rank:',rank)
            if rank == 0:
                generate_index = np.argsort((atom_list[i,:l,:16]-refer_atom_list[i,:l,:16] -\
                                             x_atom[i,:l,:16]).max(-1))[-1-top_idx]
            atom_symbol_generated = np.argsort(atom_list[i,generate_index,:16]-\
                                                    refer_atom_list[i,generate_index,:16] -\
                                                    x_atom[i,generate_index,:16])[-1-rank]
            if atom_symbol_generated==x_atom[i,generate_index,:16].argmax(-1):
#                 print('生成了相同元素，生成下一个元素……')
                rank += 1
                continue
            generate_rdkit_index = smiles_to_rdkit_list[cano_smiles][generate_index]
            if np.sort(atom_list[i,generate_index,:16]-\
                refer_atom_list[i,generate_index,:16] -\
                x_atom[i,generate_index,:16])[-1-rank]<confidence_threshold:
#                 print(f'原子位{generate_rdkit_index}生成{symbol_list[atom_symbol_generated]}元素的置信度小于{confidence_threshold}，寻找下一个原子位……')
                top_idx += 1
                rank = 0
                continue
#             if symbol_to_rdkit[atom_symbol_generated]==6:
#                 print('生成了不推荐的C元素')
#                 rank += 1
#                 continue
            mol.GetAtomWithIdx(int(generate_rdkit_index)).SetAtomicNum(symbol_to_rdkit[atom_symbol_generated])
            print_mol = mol
            try:
                Chem.SanitizeMol(mol)
                if first_run_flag == True:
                    success_validity += 1
                total[flag] += 1
                if Chem.MolToSmiles(clear_atom_map(print_mol))==y_smiles[i]:
                    success[flag] +=1
#                     print('Congratulations!', success, total)
                    success_smiles.append(Chem.MolToSmiles(clear_atom_map(print_mol)))
                mol_init = mol_with_atom_index(Chem.MolFromSmiles(smiles[i]))
#                 print("修改前的分子：", smiles[i])
#                 display(mol_init)
                modified_smiles.append(Chem.MolToSmiles(clear_atom_map(print_mol)))
#                 print(f"将第{generate_rdkit_index}个原子修改为{symbol_list[atom_symbol_generated]}的分子：", Chem.MolToSmiles(clear_atom_map(print_mol)))
#                 display(mol_with_atom_index(mol))
                mol_y = mol_with_atom_index(Chem.MolFromSmiles(y_smiles[i]))
#                 print("高活性分子：", y_smiles[i])
#                 display(mol_y)
                rank += 1
                flag += 1
            except:
#                 print(f"第{generate_rdkit_index}个原子符号修改为{symbol_list[atom_symbol_generated]}不符合规范，生成下一个元素……")
                validity_mask[i,generate_index,atom_symbol_generated] = 1
                rank += 1
                first_run_flag = False
    return success_smiles, modified_smiles, success, total, success_reconstruction, success_validity, validity_mask

def modify_bonds(smiles, x_atom, bond_neighbor, atom_list, bond_list, y_smiles, smiles_to_rdkit_list):
    x_atom = x_atom.cpu().detach().numpy()
    bond_neighbor = bond_neighbor.cpu().detach().numpy()
    atom_list = atom_list.cpu().detach().numpy()
    bond_list = bond_list.cpu().detach().numpy()
    modified_smiles = []
    for i in range(len(bond_neighbor)):
        l = (bond_neighbor[i].sum(-1).sum(-1)!=0).sum(-1)
        bond_type_sorted = np.argsort(bond_list[i,:l,:,:4], axis=-1)
        bond_type_generated_sorted = np.argsort(bond_list[i,:l,:,:4], axis=-1)
        generate_confidence_sorted = np.sort(bond_list[i,:l,:,:4], axis=-1)
        rank = 0
        top_idx = 0
        flag = 0
        while not flag==3:
            cano_smiles = Chem.MolToSmiles(Chem.MolFromSmiles(smiles[i]))
            if np.sum((bond_type_sorted[i,:,-1]!=bond_type_generated_sorted[:,:,-1-rank]).astype(int))==0:
                rank += 1
                top_idx = 0
            print('i:',i,'top_idx:', top_idx, 'rank:',rank)
            bond_type = bond_type_sorted[i,:,-1]
            bond_type_generated = bond_type_generated_sorted[:,:,-1-rank]
            generate_confidence = generate_confidence_sorted[:,:,-1-rank]
#             print(np.sort(generate_confidence + \
#                                     (atom_symbol!=atom_symbol_generated).astype(int), axis=-1))
            generate_index = np.argsort(generate_confidence + 
                                (bond_type!=bond_type_generated).astype(int), axis=-1)[-1-top_idx]
            bond_type_generated_one = bond_type_generated[generate_index]
            mol = mol_with_atom_index(Chem.MolFromSmiles(smiles[i]))
            if generate_index >= len(smiles_to_rdkit_list[cano_smiles]):
                top_idx += 1
                continue
            generate_rdkit_index = smiles_to_rdkit_list[cano_smiles][generate_index]
            mol.GetBondWithIdx(int(generate_rdkit_index)).SetBondType(bond_type_generated_one)
            try:
                Chem.SanitizeMol(mol)
                mol_init = mol_with_atom_index(Chem.MolFromSmiles(smiles[i]))
                print("修改前的分子：")
                display(mol_init)
                modified_smiles.append(mol)
                print(f"将第{generate_rdkit_index}个键修改为{atom_symbol_generated}的分子：")
                display(mol)
                mol = mol_with_atom_index(Chem.MolFromSmiles(y_smiles[i]))
                print("高活性分子：")
                display(mol)
                rank += 1
                flag += 1
            except:
                print(f"第{generate_rdkit_index}个原子符号修改为{atom_symbol_generated}不符合规范")
                top_idx += 1
    return modified_smiles
        
def eval(model, amodel, gmodel, dataset, topn=1, output_feature=False, generate=False, modify_atom=True,return_GRN_loss=False, viz=False):
    model.eval()
    amodel.eval()
    gmodel.eval()
    predict_list = []
    test_MSE_list = []
    r2_list = []
    valList = np.arange(0,dataset.shape[0])
    batch_list = []
    feature_list = []
    d_list = []
    success = [0 for i in range(topn)]
    total = [0 for i in range(topn)]
    generated_smiles = []
    success_smiles = []
    success_reconstruction = 0
    success_validity = 0
    reconstruction_loss, one_hot_loss, interger_loss, binary_loss = [0,0,0,0]
    
# #     取dataset中排序后的第k个
#     sorted_dataset = dataset.sort_values(by=tasks[0],ascending=False)
#     k_df = sorted_dataset.iloc[[k-1]]
#     k_smiles = k_df['cano_smiles'].values
#     k_value = k_df[tasks[0]].values.astype(float)    
    
    for i in range(0, dataset.shape[0], batch_size):
        batch = valList[i:i+batch_size]
        batch_list.append(batch) 
#     print(batch_list)
    for counter, batch in enumerate(batch_list):
#         print(type(batch))
        batch_df = dataset.loc[batch,:]
        smiles_list = batch_df.cano_smiles.values
        matched_smiles_list = smiles_list
#         print(batch_df)
        y_val = batch_df[tasks[0]].values.astype(float)
#         print(type(y_val))
        
        x_atom, x_bonds, x_atom_index, x_bond_index, x_mask, smiles_to_rdkit_list = get_smiles_array(matched_smiles_list,feature_dicts)
        x_atom = torch.Tensor(x_atom)
        x_bonds = torch.Tensor(x_bonds)
        x_bond_index = torch.cuda.LongTensor(x_bond_index)
        bond_neighbor = [x_bonds[i][x_bond_index[i]] for i in range(len(batch_df))]
        bond_neighbor = torch.stack(bond_neighbor, dim=0)
        
        lamda=10**-learning_rate
        activated_features, mol_feature = model(torch.Tensor(x_atom),torch.Tensor(x_bonds),torch.cuda.LongTensor(x_atom_index),torch.cuda.LongTensor(x_bond_index),torch.Tensor(x_mask),output_activated_features=True)
#         mol_feature = torch.div(mol_feature, torch.norm(mol_feature, dim=-1, keepdim=True)+1e-9)
#         activated_features = torch.div(activated_features, torch.norm(activated_features, dim=-1, keepdim=True)+1e-9)
        eps_adv, d_adv, vat_loss, mol_prediction = perturb_feature(mol_feature, amodel, alpha=1, lamda=lamda)
#         print(mol_feature,d_adv)
        atom_list, bond_list = gmodel(torch.Tensor(x_atom),torch.Tensor(x_bonds),
                                      torch.cuda.LongTensor(x_atom_index),torch.cuda.LongTensor(x_bond_index),
                                      torch.Tensor(x_mask),mol_feature=mol_feature+d_adv/(1e-6),activated_features=activated_features)
        refer_atom_list, refer_bond_list = gmodel(torch.Tensor(x_atom),torch.Tensor(x_bonds),torch.cuda.LongTensor(x_atom_index),torch.cuda.LongTensor(x_bond_index),torch.Tensor(x_mask),mol_feature=mol_feature,activated_features=activated_features)
        if generate:
            if modify_atom:
                success_smiles_batch, modified_smiles, success_batch, total_batch, reconstruction, validity, validity_mask = modify_atoms(matched_smiles_list, x_atom, 
                            bond_neighbor, atom_list, bond_list,smiles_list,smiles_to_rdkit_list,
                                                     refer_atom_list, refer_bond_list,topn=topn,viz=viz)
            else:
                modified_smiles = modify_bonds(matched_smiles_list, x_atom, bond_neighbor, atom_list, bond_list,smiles_list,smiles_to_rdkit_list)
            generated_smiles.extend(modified_smiles)
            success_smiles.extend(success_smiles_batch)
#             for n in range(topn):
#                 success[n] += success_batch[n]
#                 total[n] += total_batch[n]
#                 print('congratulations:',success,total)
            success_reconstruction += reconstruction
            success_validity += validity
            reconstruction_loss, one_hot_loss, interger_loss, binary_loss = generate_loss_function(refer_atom_list, x_atom, refer_bond_list, bond_neighbor, validity_mask, atom_list, bond_list)
        d = d_adv.cpu().detach().numpy().tolist()
        d_list.extend(d)
        mol_feature_output = mol_feature.cpu().detach().numpy().tolist()
        feature_list.extend(mol_feature_output)
#         MAE = F.l1_loss(mol_prediction, torch.Tensor(y_val).view(-1,1), reduction='none')   
#         print(type(mol_prediction))
        
        MSE = F.mse_loss(mol_prediction, torch.Tensor(y_val).view(-1,1), reduction='none')
#         r2 = caculate_r2(mol_prediction, torch.Tensor(y_val).view(-1,1))
# #         r2_list.extend(r2.cpu().detach().numpy())
#         if r2!=r2:
#             r2 = torch.tensor(0)
#         r2_list.append(r2.item())
#         predict_list.extend(mol_prediction.cpu().detach().numpy())
#         print(x_mask[:2],atoms_prediction.shape, mol_prediction,MSE)
        predict_list.extend(mol_prediction.cpu().detach().numpy())
#         test_MAE_list.extend(MAE.data.squeeze().cpu().numpy())
        test_MSE_list.extend(MSE.data.view(-1,1).cpu().numpy())
#     print(r2_list)
    if generate:
        generated_num = len(generated_smiles)
        eval_num = len(dataset)
        unique = generated_num
        novelty = generated_num
        for i in range(generated_num):
            for j in range(generated_num-i-1):
                if generated_smiles[i]==generated_smiles[i+j+1]:
                    unique -= 1
            for k in range(eval_num):
                if generated_smiles[i]==dataset['smiles'].values[k]:
                    novelty -= 1
        unique_rate = unique/(generated_num+1e-9)
        novelty_rate = novelty/(generated_num+1e-9)
#         print(f'successfully/total generated molecules =', {f'Top-{i+1}': f'{success[i]}/{total[i]}' for i in range(topn)})
        return success_reconstruction/len(dataset), success_validity/len(dataset), unique_rate, novelty_rate, success_smiles, generated_smiles, caculate_r2(predict_list,dataset[tasks[0]].values.astype(float).tolist()),np.array(test_MSE_list).mean(),predict_list
    if return_GRN_loss:
        return d_list, feature_list,caculate_r2(predict_list,dataset[tasks[0]].values.astype(float).tolist()),np.array(test_MSE_list).mean(),predict_list,reconstruction_loss, one_hot_loss, interger_loss,binary_loss
    if output_feature:
        return d_list, feature_list,caculate_r2(predict_list,dataset[tasks[0]].values.astype(float).tolist()),np.array(test_MSE_list).mean(),predict_list
    return caculate_r2(predict_list,dataset[tasks[0]].values.astype(float).tolist()),np.array(test_MSE_list).mean(),predict_list

epoch = 0
max_epoch = 1000
batch_size = 10
patience = 100
stopper = EarlyStopping(mode='higher', patience=patience, filename=model_file + '_model.pth')
stopper_afse = EarlyStopping(mode='higher', patience=patience, filename=model_file + '_amodel.pth')
stopper_generate = EarlyStopping(mode='higher', patience=patience, filename=model_file + '_gmodel.pth')

In [14]:
import datetime
from tensorboardX import SummaryWriter
now = datetime.datetime.now().strftime('%b%d_%H-%M-%S')
# if os.path.isdir(log_dir):
#     for files in os.listdir(log_dir):
#         os.remove(log_dir+"/"+files)
#     os.rmdir(log_dir)
logger = SummaryWriter(log_dir)
print(log_dir)

# train_f_list=[]
# train_mse_list=[]
# train_r2_list=[]
# test_f_list=[]
# test_mse_list=[]
# test_r2_list=[]
# val_f_list=[]
# val_mse_list=[]
# val_r2_list=[]
# epoch_list=[]
# train_predict_list=[]
# test_predict_list=[]
# val_predict_list=[]
# train_y_list=[]
# test_y_list=[]
# val_y_list=[]
# train_d_list=[]
# test_d_list=[]
# val_d_list=[]

epoch = 155
stopper.load_checkpoint(model)
stopper_afse.load_checkpoint(amodel)
stopper_generate.load_checkpoint(gmodel)
optimizer_list = [optimizer, optimizer_AFSE, optimizer_GRN]
max_epoch = 1000
while epoch < max_epoch:
    train(model, amodel, gmodel, train_df, test_df, optimizer_list, loss_function, epoch)
#     print(train_df.shape,test_df.shape)
    train_d, train_f, train_r2, train_MSE, train_predict, reconstruction_loss, one_hot_loss, interger_loss,binary_loss = eval(model, amodel, gmodel, train_df,output_feature=True,return_GRN_loss=True)
    train_predict = np.array(train_predict)
    train_WTI = weighted_top_index(train_df, train_predict, len(train_df))
    train_tau, _ = scipy.stats.kendalltau(train_predict,train_df[tasks[0]].values.astype(float).tolist())
    val_d, val_f, val_r2, val_MSE, val_predict, val_reconstruction_loss, val_one_hot_loss, val_interger_loss,val_binary_loss = eval(model, amodel, gmodel, val_df,output_feature=True,return_GRN_loss=True)
    val_predict = np.array(val_predict)
    val_WTI = weighted_top_index(val_df, val_predict, len(val_df))
    val_AP = AP(val_df, val_predict, len(val_df))
    val_tau, _ = scipy.stats.kendalltau(val_predict,val_df[tasks[0]].values.astype(float).tolist())
    
    test_r2_a, test_MSE_a, test_predict_a = eval(model, amodel, gmodel, test_df[:test_active])
    test_d, test_f, test_r2, test_MSE, test_predict = eval(model, amodel, gmodel, test_df,output_feature=True)
    test_predict = np.array(test_predict)
    test_WTI = weighted_top_index(test_df, test_predict, test_active)
#     test_AP = AP(test_df, test_predict, test_active)
    test_tau, _ = scipy.stats.kendalltau(test_predict,test_df[tasks[0]].values.astype(float).tolist())
    
    k_list = [int(len(test_df)*0.01),int(len(test_df)*0.03),int(len(test_df)*0.1),10,30,100]
    topk_list =[]
    false_positive_rate_list = []
    for k in k_list:
        a,b = topk_acc_recall(test_df, test_predict, k, test_active, False, epoch)
        topk_list.append(a)
        false_positive_rate_list.append(b)
    
    epoch = epoch + 1
    global_step = epoch * int(np.max([len(train_df),len(test_df)])/batch_size)
    logger.add_scalar('train/r2', train_r2, global_step)
    logger.add_scalar('train/RMSE', train_MSE**0.5, global_step)
    logger.add_scalar('train/Tau', train_tau, global_step)
    logger.add_scalar('val/WTI', val_WTI, global_step)
    logger.add_scalar('val/AP', val_AP, global_step)
    logger.add_scalar('val/r2', val_r2, global_step)
    logger.add_scalar('val/RMSE', val_MSE**0.5, global_step)
    logger.add_scalar('val/Tau', val_tau, global_step)
#     logger.add_scalar('test/TAP', test_AP, global_step)
    logger.add_scalar('test/r2', test_r2_a, global_step)
    logger.add_scalar('test/RMSE', test_MSE_a**0.5, global_step)
    logger.add_scalar('test/Tau', test_tau, global_step)
    logger.add_scalar('val/GRN', reconstruction_loss, global_step)
    logger.add_scalar('val/GRN_one_hot', one_hot_loss, global_step)
    logger.add_scalar('val/GRN_interger', interger_loss, global_step)
    logger.add_scalar('val/GRN_binary', binary_loss, global_step)
    logger.add_scalar('test/EF0.01', topk_list[0], global_step)
    logger.add_scalar('test/EF0.03', topk_list[1], global_step)
    logger.add_scalar('test/EF0.1', topk_list[2], global_step)
    logger.add_scalar('test/EF10', topk_list[3], global_step)
    logger.add_scalar('test/EF30', topk_list[4], global_step)
    logger.add_scalar('test/EF100', topk_list[5], global_step)
    
#     train_mse_list.append(train_MSE**0.5)
#     train_r2_list.append(train_r2)
#     val_mse_list.append(val_MSE**0.5)  
#     val_r2_list.append(val_r2)
#     train_f_list.append(train_f)
#     val_f_list.append(val_f)
#     test_f_list.append(test_f)
#     epoch_list.append(epoch)
#     train_predict_list.append(train_predict.flatten())
#     test_predict_list.append(test_predict.flatten())
#     val_predict_list.append(val_predict.flatten())
#     train_y_list.append(train_df[tasks[0]].values)
#     val_y_list.append(val_df[tasks[0]].values)
#     test_y_list.append(test_df[tasks[0]].values)
#     train_d_list.append(train_d)
#     val_d_list.append(val_d)
#     test_d_list.append(test_d)

    stop_index = - val_MSE**0.5 + val_tau
    early_stop = stopper.step(stop_index, model)
    early_stop = stopper_afse.step(stop_index, amodel, if_print=False)
    early_stop = stopper_generate.step(stop_index, gmodel, if_print=False)
#     print('epoch {:d}/{:d}, validation {} {:.4f}, {} {:.4f},best validation {r2} {:.4f}'.format(epoch, total_epoch, 'r2', val_r2, 'mse:',val_MSE, stopper.best_score))
    print('Epoch:',epoch, 'Step:', global_step, 'Index:%.4f'%stop_index, 'R2:%.4f'%train_r2,'%.4f'%val_r2,'%.4f'%test_r2_a, 'RMSE:%.4f'%train_MSE**0.5, '%.4f'%val_MSE**0.5, 
          '%.4f'%test_MSE_a**0.5, 'Tau:%.4f'%train_tau,'%.4f'%val_tau,'%.4f'%test_tau)#, 'Tau:%.4f'%val_tau,'%.4f'%test_tau,'GRN:%.4f'%reconstruction_loss,'%.4f'%val_reconstruction_loss
    if early_stop:
        continue


log/3_GAFSE_EC50_Q99500_1_100_run_0


/data/yinmingyue/anaconda3/envs/env/lib/python3.7/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


Epoch: 156 Step: 16380 Index:-0.0931 R2:0.6727 0.5654 0.5218 RMSE:0.5038 0.5922 0.6209 Tau:0.5966 0.4991 0.2453
Epoch: 157 Step: 16485 Index:-0.0201 R2:0.7019 0.5935 0.5399 RMSE:0.4401 0.5393 0.5634 Tau:0.6102 0.5192 0.2579
EarlyStopping counter: 1 out of 100
Epoch: 158 Step: 16590 Index:-0.0230 R2:0.7126 0.5997 0.4777 RMSE:0.4407 0.5378 0.6020 Tau:0.6100 0.5148 0.2465
EarlyStopping counter: 2 out of 100
Epoch: 159 Step: 16695 Index:-0.0786 R2:0.7054 0.5994 0.5355 RMSE:0.5121 0.5934 0.6299 Tau:0.6150 0.5148 0.2491
Epoch: 160 Step: 16800 Index:0.0079 R2:0.7139 0.6143 0.5125 RMSE:0.4374 0.5313 0.5804 Tau:0.6161 0.5392 0.2570
EarlyStopping counter: 1 out of 100
Epoch: 161 Step: 16905 Index:-0.0038 R2:0.7109 0.6205 0.5602 RMSE:0.4424 0.5349 0.5506 Tau:0.6187 0.5311 0.2610
EarlyStopping counter: 2 out of 100
Epoch: 162 Step: 17010 Index:-0.0247 R2:0.7136 0.5880 0.4846 RMSE:0.4362 0.5439 0.5954 Tau:0.6160 0.5192 0.2466
EarlyStopping counter: 3 out of 100
Epoch: 163 Step: 17115 Index:-0.0115 

EarlyStopping counter: 35 out of 100
Epoch: 213 Step: 22365 Index:-0.0508 R2:0.7455 0.6029 0.5013 RMSE:0.4362 0.5585 0.6124 Tau:0.6392 0.5076 0.2539
EarlyStopping counter: 36 out of 100
Epoch: 214 Step: 22470 Index:0.0018 R2:0.7708 0.6145 0.4914 RMSE:0.3875 0.5259 0.5860 Tau:0.6602 0.5277 0.2542
EarlyStopping counter: 37 out of 100
Epoch: 215 Step: 22575 Index:-0.0838 R2:0.7568 0.5928 0.5225 RMSE:0.4884 0.6006 0.6501 Tau:0.6554 0.5168 0.2501
EarlyStopping counter: 38 out of 100
Epoch: 216 Step: 22680 Index:-0.0361 R2:0.7628 0.5738 0.4532 RMSE:0.4029 0.5573 0.6178 Tau:0.6525 0.5212 0.2508
EarlyStopping counter: 39 out of 100
Epoch: 217 Step: 22785 Index:-0.0120 R2:0.7641 0.6065 0.4754 RMSE:0.3902 0.5407 0.5906 Tau:0.6568 0.5287 0.2542
EarlyStopping counter: 40 out of 100
Epoch: 218 Step: 22890 Index:-0.0575 R2:0.7678 0.6068 0.4827 RMSE:0.4255 0.5726 0.6019 Tau:0.6582 0.5151 0.2507
EarlyStopping counter: 41 out of 100
Epoch: 219 Step: 22995 Index:-0.0077 R2:0.7663 0.6060 0.4491 RMSE:0.39

EarlyStopping counter: 26 out of 100
Epoch: 269 Step: 28245 Index:-0.0438 R2:0.8109 0.5726 0.3685 RMSE:0.3569 0.5545 0.6539 Tau:0.6928 0.5107 0.2406
EarlyStopping counter: 27 out of 100
Epoch: 270 Step: 28350 Index:-0.0268 R2:0.8173 0.6018 0.4358 RMSE:0.3746 0.5592 0.6499 Tau:0.7018 0.5324 0.2519
EarlyStopping counter: 28 out of 100
Epoch: 271 Step: 28455 Index:-0.0520 R2:0.8096 0.5813 0.3933 RMSE:0.4154 0.5855 0.6958 Tau:0.6923 0.5335 0.2407
EarlyStopping counter: 29 out of 100
Epoch: 272 Step: 28560 Index:-0.0472 R2:0.8110 0.5805 0.3835 RMSE:0.3571 0.5657 0.6511 Tau:0.6941 0.5185 0.2489
EarlyStopping counter: 30 out of 100
Epoch: 273 Step: 28665 Index:-0.0473 R2:0.7914 0.5791 0.4562 RMSE:0.3675 0.5555 0.6041 Tau:0.6849 0.5083 0.2525
EarlyStopping counter: 31 out of 100
Epoch: 274 Step: 28770 Index:-0.0650 R2:0.8034 0.5640 0.3341 RMSE:0.3688 0.5682 0.7015 Tau:0.6816 0.5032 0.2470
EarlyStopping counter: 32 out of 100
Epoch: 275 Step: 28875 Index:-0.0002 R2:0.8156 0.6113 0.4304 RMSE:0.3

...

EarlyStopping counter: 518 out of 100
Epoch: 761 Step: 79905 Index:-140.6108 R2:0.9217 0.0183 0.0015 RMSE:0.2346 141.0347 142.7084 Tau:0.8032 0.4239 0.2838
EarlyStopping counter: 519 out of 100
Epoch: 762 Step: 80010 Index:-165.3785 R2:0.9336 0.0255 0.0028 RMSE:0.2121 165.8283 172.6195 Tau:0.8266 0.4498 0.2816
EarlyStopping counter: 520 out of 100
Epoch: 763 Step: 80115 Index:-193.4369 R2:0.9372 0.0344 0.0039 RMSE:0.2462 193.8802 204.7905 Tau:0.8259 0.4433 0.2885
EarlyStopping counter: 521 out of 100
Epoch: 764 Step: 80220 Index:-203.0954 R2:0.9205 0.0367 0.0042 RMSE:0.2330 203.5367 214.5030 Tau:0.8104 0.4413 0.2867
EarlyStopping counter: 522 out of 100
Epoch: 765 Step: 80325 Index:-0.5904 R2:0.7923 0.3356 0.1357 RMSE:0.4810 1.0892 1.4880 Tau:0.6821 0.4988 0.2875
EarlyStopping counter: 523 out of 100
Epoch: 766 Step: 80430 Index:-0.3968 R2:0.9254 0.4211 0.1494 RMSE:0.2309 0.9095 1.4792 Tau:0.8116 0.5127 0.2948
EarlyStopping counter: 524 out of 100
Epoch: 767 Step: 80535 Index:-0.3847 R

EarlyStopping counter: 572 out of 100
Epoch: 815 Step: 85575 Index:-3.0679 R2:0.9239 0.0021 0.0130 RMSE:0.2924 3.5075 6.1311 Tau:0.8142 0.4396 0.2836
EarlyStopping counter: 573 out of 100
Epoch: 816 Step: 85680 Index:-3.7492 R2:0.9273 0.0001 0.0077 RMSE:0.2596 4.1861 7.6940 Tau:0.8173 0.4369 0.2857
EarlyStopping counter: 574 out of 100
Epoch: 817 Step: 85785 Index:-4.9421 R2:0.8986 0.0030 0.0041 RMSE:0.2571 5.3909 11.0936 Tau:0.7807 0.4488 0.2905
EarlyStopping counter: 575 out of 100
Epoch: 818 Step: 85890 Index:-8.0625 R2:0.9397 0.0092 0.0017 RMSE:0.2033 8.5262 20.7326 Tau:0.8344 0.4637 0.2934
EarlyStopping counter: 576 out of 100
Epoch: 819 Step: 85995 Index:-10.1318 R2:0.9391 0.0137 0.0011 RMSE:0.2309 10.5792 26.8971 Tau:0.8325 0.4474 0.2874
EarlyStopping counter: 577 out of 100
Epoch: 820 Step: 86100 Index:-14.7711 R2:0.9317 0.0200 0.0006 RMSE:0.2130 15.2352 40.2011 Tau:0.8286 0.4641 0.2914
EarlyStopping counter: 578 out of 100
Epoch: 821 Step: 86205 Index:-25.3168 R2:0.9435 0.0258

EarlyStopping counter: 625 out of 100
Epoch: 868 Step: 91140 Index:-366.2326 R2:0.9347 0.0501 0.0055 RMSE:0.2136 366.6898 365.4844 Tau:0.8333 0.4573 0.2952
EarlyStopping counter: 626 out of 100
Epoch: 869 Step: 91245 Index:-378.0135 R2:0.9326 0.0505 0.0056 RMSE:0.2093 378.4762 373.6078 Tau:0.8242 0.4627 0.2940
EarlyStopping counter: 627 out of 100
Epoch: 870 Step: 91350 Index:-379.4159 R2:0.9226 0.0508 0.0057 RMSE:0.3836 379.8878 372.2779 Tau:0.8100 0.4719 0.2972
EarlyStopping counter: 628 out of 100
Epoch: 871 Step: 91455 Index:-112.6494 R2:0.9399 0.0001 0.0083 RMSE:0.2356 113.0971 9.9012 Tau:0.8312 0.4478 0.2939
EarlyStopping counter: 629 out of 100
Epoch: 872 Step: 91560 Index:-160.1383 R2:0.9417 0.0011 0.0004 RMSE:0.2056 160.5875 68.3909 Tau:0.8331 0.4491 0.2959
EarlyStopping counter: 630 out of 100
Epoch: 873 Step: 91665 Index:-195.4941 R2:0.9322 0.0064 0.0001 RMSE:0.3026 195.9221 155.4301 Tau:0.8173 0.4280 0.2917
EarlyStopping counter: 631 out of 100
Epoch: 874 Step: 91770 Index:

EarlyStopping counter: 679 out of 100
Epoch: 922 Step: 96810 Index:-410.3907 R2:0.9336 0.0590 0.0055 RMSE:0.2189 410.8548 443.9027 Tau:0.8353 0.4641 0.2994
EarlyStopping counter: 680 out of 100
Epoch: 923 Step: 96915 Index:-417.2060 R2:0.9129 0.0581 0.0056 RMSE:0.2773 417.6697 445.5724 Tau:0.8160 0.4637 0.2943
EarlyStopping counter: 681 out of 100
Epoch: 924 Step: 97020 Index:-438.9629 R2:0.9205 0.0572 0.0057 RMSE:0.3032 439.4294 456.8801 Tau:0.8078 0.4665 0.3003
EarlyStopping counter: 682 out of 100
Epoch: 925 Step: 97125 Index:-436.5207 R2:0.9447 0.0569 0.0057 RMSE:0.1915 436.9820 454.3045 Tau:0.8408 0.4614 0.2930
EarlyStopping counter: 683 out of 100
Epoch: 926 Step: 97230 Index:-460.6966 R2:0.9485 0.0563 0.0058 RMSE:0.1946 461.1583 469.5413 Tau:0.8452 0.4617 0.2993
EarlyStopping counter: 684 out of 100
Epoch: 927 Step: 97335 Index:-466.9315 R2:0.9469 0.0560 0.0058 RMSE:0.1885 467.3795 473.6671 Tau:0.8429 0.4481 0.2921
EarlyStopping counter: 685 out of 100
Epoch: 928 Step: 97440 Ind

EarlyStopping counter: 732 out of 100
Epoch: 975 Step: 102375 Index:-541.5188 R2:0.9481 0.0509 0.0058 RMSE:0.2117 541.9853 510.5454 Tau:0.8514 0.4665 0.2940
EarlyStopping counter: 733 out of 100
Epoch: 976 Step: 102480 Index:-548.9226 R2:0.9424 0.0509 0.0058 RMSE:0.1921 549.3785 516.4780 Tau:0.8435 0.4559 0.2936
EarlyStopping counter: 734 out of 100
Epoch: 977 Step: 102585 Index:-560.4437 R2:0.9424 0.0506 0.0058 RMSE:0.2268 560.9088 525.3286 Tau:0.8394 0.4651 0.2989
EarlyStopping counter: 735 out of 100
Epoch: 978 Step: 102690 Index:-267.4317 R2:0.9432 0.0050 0.0003 RMSE:0.1990 267.9036 188.5782 Tau:0.8406 0.4719 0.2947
EarlyStopping counter: 736 out of 100
Epoch: 979 Step: 102795 Index:-403.6692 R2:0.9457 0.0355 0.0038 RMSE:0.1885 404.1360 383.2224 Tau:0.8403 0.4668 0.2953
EarlyStopping counter: 737 out of 100
Epoch: 980 Step: 102900 Index:-483.2333 R2:0.9529 0.0460 0.0048 RMSE:0.2004 483.6848 449.3849 Tau:0.8532 0.4515 0.2941
EarlyStopping counter: 738 out of 100
Epoch: 981 Step: 103

In [15]:
stopper.load_checkpoint(model)
stopper_afse.load_checkpoint(amodel)
stopper_generate.load_checkpoint(gmodel)
    
test_r2, test_MSE, test_predict = eval(model, amodel, gmodel, test_df)
test_r2_a, test_MSE_a, test_predict_a = eval(model, amodel, gmodel, test_df[:test_active])
test_r2_ina, test_MSE_ina, test_predict_ina = eval(model, amodel, gmodel, test_df[test_active:].reset_index(drop=True))
    
test_predict = np.array(test_predict)
test_tau, _ = scipy.stats.kendalltau(test_predict,test_df[tasks[0]].values.astype(float).tolist())

k_list = [int(len(test_df)*0.01),int(len(test_df)*0.05),int(len(test_df)*0.1),int(len(test_df)*0.15),int(len(test_df)*0.2),int(len(test_df)*0.25),
          int(len(test_df)*0.3),int(len(test_df)*0.4),int(len(test_df)*0.5),50,100,150,200,250,300]
topk_list =[]
false_positive_rate_list = []
for k in k_list:
    a,b = topk_acc_recall(test_df, test_predict, k, test_active, False, epoch)
    topk_list.append(a)
    false_positive_rate_list.append(b)
WTI = weighted_top_index(test_df, test_predict, test_active)
ap = AP(test_df, test_predict, test_active)


In [16]:
print(' epoch:',epoch,'r2:%.4f'%test_r2_a,'RMSE:%.4f'%test_MSE_a**0.5,'WTI:%.4f'%WTI,'AP:%.4f'%ap,'Tau:%.4f'%test_tau,'\n','\n',
      'Top-1:%.4f'%topk_list[0],'Top-1-fp:%.4f'%false_positive_rate_list[0],'\n',
      'Top-5:%.4f'%topk_list[1],'Top-5-fp:%.4f'%false_positive_rate_list[1],'\n',
      'Top-10:%.4f'%topk_list[2],'Top-10-fp:%.4f'%false_positive_rate_list[2],'\n',
      'Top-15:%.4f'%topk_list[3],'Top-15-fp:%.4f'%false_positive_rate_list[3],'\n',
      'Top-20:%.4f'%topk_list[4],'Top-20-fp:%.4f'%false_positive_rate_list[4],'\n',
      'Top-25:%.4f'%topk_list[5],'Top-25-fp:%.4f'%false_positive_rate_list[5],'\n',
      'Top-30:%.4f'%topk_list[6],'Top-30-fp:%.4f'%false_positive_rate_list[6],'\n',
      'Top-40:%.4f'%topk_list[7],'Top-40-fp:%.4f'%false_positive_rate_list[7],'\n',
      'Top-50:%.4f'%topk_list[8],'Top-50-fp:%.4f'%false_positive_rate_list[8],'\n','\n',
      'Top50:%.4f'%topk_list[9],'Top50-fp:%.4f'%false_positive_rate_list[9],'\n',
      'Top100:%.4f'%topk_list[10],'Top100-fp:%.4f'%false_positive_rate_list[10],'\n',
      'Top150:%.4f'%topk_list[11],'Top150-fp:%.4f'%false_positive_rate_list[11],'\n',
      'Top200:%.4f'%topk_list[12],'Top200-fp:%.4f'%false_positive_rate_list[12],'\n',
      'Top250:%.4f'%topk_list[13],'Top250-fp:%.4f'%false_positive_rate_list[13],'\n',
      'Top300:%.4f'%topk_list[14],'Top300-fp:%.4f'%false_positive_rate_list[14],'\n')

 epoch: 1000 r2:0.5041 RMSE:0.5764 WTI:0.3370 AP:0.2844 Tau:0.2680 
 
 Top-1:0.2000 Top-1-fp:0.8000 
 Top-5:0.3077 Top-5-fp:0.6731 
 Top-10:0.3600 Top-10-fp:0.6571 
 Top-15:0.4800 Top-15-fp:0.6943 
 Top-20:0.6000 Top-20-fp:0.7143 
 Top-25:0.6400 Top-25-fp:0.7567 
 Top-30:0.7100 Top-30-fp:0.7746 
 Top-40:0.8300 Top-40-fp:0.8029 
 Top-50:0.9200 Top-50-fp:0.8251 
 
 Top50:0.3000 Top50-fp:0.6800 
 Top100:0.3200 Top100-fp:0.6800 
 Top150:0.4600 Top150-fp:0.6933 
 Top200:0.5800 Top200-fp:0.7100 
 Top250:0.6300 Top250-fp:0.7480 
 Top300:0.7100 Top300-fp:0.7633 



In [17]:
# print('target_file:',train_filename)
# print('inactive_file:',test_filename)
# np.savez(result_dir, epoch_list, train_f_list, train_d_list, 
#          train_predict_list, train_y_list, val_f_list, val_d_list, val_predict_list, val_y_list, test_f_list, 
#          test_d_list, test_predict_list, test_y_list)
# sim_space = np.load(result_dir+'.npz')
# print(sim_space['arr_10'].shape)

In [18]:
# loss = loss_function(mol_prediction,y)
#             loss.backward(retain_graph=True)
#             optimizer_AFSE.zero_grad()
#             punish_lr = torch.norm(torch.mean(eps.grad,0))

#         init_lr = 1e-4
#         max_lr = 10**-(init_lr-1)
#         conv_lr = conv_lr - conv_lr**2 + 0.1 * punish_lr
#         if conv_lr < max_lr:
#             for param_group in optimizer_AFSE.param_groups:
#                 param_group["lr"] = conv_lr.detach()
#                 AFSE_lr = conv_lr    
#         else:
#             for param_group in optimizer_AFSE.param_groups:
#                 param_group["lr"] = max_lr
#                 AFSE_lr = max_lr
# epoch: 512 r2:0.5619 RMSE:0.7306 WTI:0.3784 AP:0.7228 Tau:0.5159 
 
#  Top-1:0.0000 Top-1-fp:0.0000 
#  Top-5:0.8571 Top-5-fp:0.0000 
#  Top-10:0.7857 Top-10-fp:0.0714 